In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
import glob
from tqdm import tqdm
from matplotlib.colors import LogNorm
import matplotlib.ticker as ticker


In [2]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371000  # Radius of earth in meters
    return c * r

def process_taxi_file(filepath):
    """Process a single taxi file and return data for analysis."""
    time_intervals = []
    distance_intervals = []
    points = []
    
    try:
        # Read the file with proper column names
        df = pd.read_csv(filepath, header=None, 
                        names=['taxi_id', 'datetime', 'longitude', 'latitude'])
        
        # Convert datetime string to datetime object
        df['datetime'] = pd.to_datetime(df['datetime'])
        
        # Sort by datetime to ensure correct order
        df = df.sort_values('datetime').reset_index(drop=True)
        
        # Extract all GPS points
        for _, row in df.iterrows():
            points.append((row['longitude'], row['latitude'], row['datetime']))
        
        # Calculate intervals between consecutive points
        for i in range(1, len(df)):
            # Time interval in minutes
            time_diff = (df.iloc[i]['datetime'] - df.iloc[i-1]['datetime']).total_seconds() / 60
            
            # Distance in meters
            dist = haversine(
                df.iloc[i-1]['longitude'], df.iloc[i-1]['latitude'],
                df.iloc[i]['longitude'], df.iloc[i]['latitude']
            )
            
            # Only add reasonable values (filter outliers)
            if 0 < time_diff < 60 and 0 < dist < 10000:  # Max 60 min and 10km
                time_intervals.append(time_diff)
                distance_intervals.append(dist)
    
    except Exception as e:
        print(f"Error processing {filepath}: {e}")
    
    return time_intervals, distance_intervals, points


In [3]:
data_dir = 'data'

# Check if data directory exists
if not os.path.exists(data_dir):
    print(f"Data directory {data_dir} not found.")

In [4]:

# Get all taxi files
taxi_files = glob.glob(os.path.join(data_dir, '*.txt'))

if not taxi_files:
    print(f"No taxi data files found in {data_dir}.")

print(f"Found {len(taxi_files)} taxi data files.")

Found 10357 taxi data files.


In [5]:

# Process data
all_time_intervals = []
all_distance_intervals = []
all_points = []

# Process each taxi file
# Adjust the number of files to process based on your computational resources
#num_files_to_process = min(1000, len(taxi_files))  # Process up to 1000 files
num_files_to_process = len(taxi_files)
 
print(f"Processing {num_files_to_process} taxi files...")
for file in tqdm(taxi_files[:num_files_to_process]):
    time_intervals, distance_intervals, points = process_taxi_file(file)
    all_time_intervals.extend(time_intervals)
    all_distance_intervals.extend(distance_intervals)
    all_points.extend(points)

print(f"Processed {len(all_time_intervals)} interval data points.")
print(f"Collected {len(all_points)} GPS points.")


Processing 10357 taxi files...


  4%|▍         | 446/10357 [01:37<28:25,  5.81it/s]  /var/folders/b1/jmw1t8k560b3ljdj7pzwl_kh0000gn/T/ipykernel_23924/354684734.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime'])
100%|██████████| 10357/10357 [3:44:08<00:00,  1.30s/it]    

Processed 13552798 interval data points.
Collected 17662984 GPS points.


In [6]:
import pickle

# Save processed data to a file
processed_data_file = 'processed_taxi_data.pkl'

with open(processed_data_file, 'wb') as f:
    pickle.dump({
        'all_time_intervals': all_time_intervals,
        'all_distance_intervals': all_distance_intervals,
        'all_points': all_points
    }, f)

print(f"Processed data saved to {processed_data_file}.")

Processed data saved to processed_taxi_data.pkl.


In [7]:
with open(processed_data_file, 'rb') as f:
    loaded_data = pickle.load(f)

# Extract data from the loaded dictionary
loaded_time_intervals = loaded_data['all_time_intervals']
loaded_distance_intervals = loaded_data['all_distance_intervals']
loaded_points = loaded_data['all_points']

print("Pickled data loaded successfully.")

Pickled data loaded successfully.


In [8]:

def create_interval_histograms(time_intervals, distance_intervals):
    """Create histograms for time and distance intervals."""
    plt.figure(figsize=(12, 5))
    
    # Time interval histogram (minutes)
    plt.subplot(1, 2, 1)
    counts, bins, _ = plt.hist(time_intervals, bins=30, range=(0, 12), 
                              density=True, alpha=0.7, color='blue')
    plt.xlabel('minutes')
    plt.ylabel('proportion')
    plt.title('(a) time interval')
    plt.grid(True, alpha=0.3)
    
    # Distance interval histogram (meters)
    plt.subplot(1, 2, 2)
    counts, bins, _ = plt.hist(distance_intervals, bins=30, range=(0, 8000), 
                              density=True, alpha=0.7, color='green')
    plt.xlabel('meters')
    plt.ylabel('proportion')
    plt.title('(b) distance interval')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('tdrive_intervals_histogram.png', dpi=300)
    plt.close()


In [9]:

def create_density_plot(all_points):
    """Create density plot of GPS points."""
    # Extract longitudes and latitudes
    longitudes = [p[0] for p in all_points]
    latitudes = [p[1] for p in all_points]
    
    # Create full Beijing overview plot
    plt.figure(figsize=(10, 8))
    
    # Use hexbin for density visualization with logarithmic color scale
    hb = plt.hexbin(longitudes, latitudes, 
                   gridsize=200, 
                   cmap='viridis', 
                   bins='log',
                   extent=[116.1, 116.8, 39.6, 40.2])
    
    # Add colorbar
    cb = plt.colorbar(hb, label='Number of GPS points')
    
    # Set labels and title
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('(a) Data overview in Beijing')
    
    # Set axis limits to match the document
    plt.xlim(116.1, 116.8)
    plt.ylim(39.6, 40.2)
    
    # Add grid
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('tdrive_density_overview.png', dpi=300)
    plt.close()
    
    # Create inner city plot (within 5th Ring Road)
    plt.figure(figsize=(10, 8))
    
    # Use hexbin for density visualization with logarithmic color scale
    hb = plt.hexbin(longitudes, latitudes, 
                   gridsize=200, 
                   cmap='viridis', 
                   bins='log',
                   extent=[116.2, 116.55, 39.8, 40.05])
    
    # Add colorbar
    cb = plt.colorbar(hb, label='Number of GPS points')
    
    # Set labels and title
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('(b) Within the 5th Ring Road of Beijing')
    
    # Set axis limits to match the document
    plt.xlim(116.2, 116.55)
    plt.ylim(39.8, 40.05)
    
    # Add grid
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('tdrive_density_inner_city.png', dpi=300)
    plt.close()

In [10]:
def process_taxi_file(filepath):
    # ... existing code ...
    for _, row in df.iterrows():
        points.append((row['longitude'], row['latitude'], row['datetime']))
    # ... rest of the code ...

In [11]:

# Convert to numpy arrays for processing
time_intervals_array = np.array(all_time_intervals)
distance_intervals_array = np.array(all_distance_intervals)

# Create interval histograms
print("Creating interval histograms...")
create_interval_histograms(time_intervals_array, distance_intervals_array)

# Create density plot
print("Creating density plots...")
create_density_plot(all_points)

# Print statistics
print("\nTime Interval Statistics (minutes):")
print(f"Count: {len(all_time_intervals)}")
print(f"Min: {np.min(time_intervals_array):.2f}")
print(f"Max: {np.max(time_intervals_array):.2f}")
print(f"Mean: {np.mean(time_intervals_array):.2f}")
print(f"Median: {np.median(time_intervals_array):.2f}")

print("\nDistance Interval Statistics (meters):")
print(f"Count: {len(all_distance_intervals)}")
print(f"Min: {np.min(distance_intervals_array):.2f}")
print(f"Max: {np.max(distance_intervals_array):.2f}")
print(f"Mean: {np.mean(distance_intervals_array):.2f}")
print(f"Median: {np.median(distance_intervals_array):.2f}")

print("\nAnalysis complete! Output images:")
print("1. tdrive_intervals_histogram.png")
print("2. tdrive_density_overview.png")
print("3. tdrive_density_inner_city.png")

Creating interval histograms...
Creating density plots...

Time Interval Statistics (minutes):
Count: 13552798
Min: 0.02
Max: 59.98
Mean: 3.63
Median: 3.92

Distance Interval Statistics (meters):
Count: 13552798
Min: 0.78
Max: 9999.97
Mean: 749.50
Median: 142.33

Analysis complete! Output images:
1. tdrive_intervals_histogram.png
2. tdrive_density_overview.png
3. tdrive_density_inner_city.png
